In [6]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, LSTM, Embedding, Concatenate, BatchNormalization
from tensorflow.keras.optimizers import Adam

import pickle
from tensorflow.keras.models import model_from_json


In [7]:
# Importamos el csv
FICHERO_DATA = 'data/data_reviews.csv'
df = pd.read_csv(FICHERO_DATA)
df.drop(columns=['expresiones'], inplace=True) # No aporta nada y el futuro df de validación no tendrá esta columna
df.head()

,reseñas,IA,longitud,longitud_promedio,palabras_unicas,signos_de_puntuación,frecuencia_pronombres,variedad_lexica,entropia_lexica,tfidf,palabras_complejas,tecnicismos,polaridad,subjetividad,coherencia
0,Necesitaba un nuevo movil y me decidí por este...,0,297,4.066667,0.733333,6,0,0.733333,5.267482,5.640761e-02,3,0,0.0,0.0,0.920000
1,Está en perfectas condiciones la batería al 10...,0,99,5.375000,1.000000,2,0,1.000000,4.000000,5.551115e-17,2,0,0.0,0.0,1.000000
2,Me llegó ayer el iPhone en perfectas condicion...,0,436,4.402439,0.695122,7,0,0.707317,5.710972,5.749596e-02,5,0,0.0,0.0,0.857143
3,Compré un iPhone 12 a este vendedor y hace un...,0,657,4.124031,0.581395,10,1,0.604651,5.880116,7.866846e-02,5,0,0.0,0.0,0.823529
4,Soy usuario de IPhone desde el principio. Teng...,0,596,4.109244,0.638655,11,0,0.655462,5.953860,7.357145e-02,4,0,0.0,0.0,0.953488


In [8]:
df.columns

Index(['reseñas', 'IA', 'longitud', 'longitud_promedio', 'palabras_unicas',
       'signos_de_puntuación', 'frecuencia_pronombres', 'variedad_lexica',
       'entropia_lexica', 'tfidf', 'palabras_complejas', 'tecnicismos',
       'polaridad', 'subjetividad', 'coherencia'],
      dtype='object')

In [9]:
# Red neuronal hibrida ya que analiza caracteristicas numericas obtenidas previamente con procesamiento del lenguaje natural y caracteristicas en texto
# mediante un Embedding y una capa LSTM para capturar relaciones temporales en el texto.

# 1. Separar características numéricas y texto
X_text = df['reseñas']
X_numerical = df.drop(columns=['reseñas', 'IA'])
y = df['IA']

# 2. Separación de los conjuntos de entrenamiento y prueba
X_train_text, X_test_text, X_train_numerical, X_test_numerical, y_train, y_test = train_test_split(
    X_text, X_numerical, y, test_size=0.2, random_state=42
)

# 3. Escalado de las características numéricas
scaler = StandardScaler()
X_train_numerical_scaled = scaler.fit_transform(X_train_numerical)
X_test_numerical_scaled = scaler.transform(X_test_numerical)

# 4. Tokenización del texto (reseñas)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['reseñas'])

# Tamaño del vocabulario
vocab_size = len(tokenizer.word_index) + 1  # Añadimos 1 porque el indexado empieza en 1
max_length = X_text.apply(lambda x: len(x.split())).max()  # Longitud máxima de las secuencias

# Convertir texto en secuencias de enteros
X_train_text_seq = tokenizer.texts_to_sequences(X_train_text)
X_test_text_seq = tokenizer.texts_to_sequences(X_test_text)

# Padding de las secuencias para que todas tengan la misma longitud
X_train_text_padded = pad_sequences(X_train_text_seq, maxlen=max_length, padding='post')
X_test_text_padded = pad_sequences(X_test_text_seq, maxlen=max_length, padding='post')

# 5. Definir el modelo más complejo
numerical_input = Input(shape=(X_train_numerical_scaled.shape[1],), name='numerical_input')
x1 = Dense(128, activation='relu')(numerical_input)
x1 = BatchNormalization()(x1)
x1 = Dropout(0.5)(x1)

text_input = Input(shape=(max_length,), name='text_input')
x2 = Embedding(input_dim=vocab_size, output_dim=128, input_length=max_length)(text_input)
x2 = LSTM(128, return_sequences=False)(x2)
x2 = Dropout(0.5)(x2)

combined = Concatenate()([x1, x2])

x = Dense(64, activation='relu')(combined)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.5)(x)

output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=[numerical_input, text_input], outputs=output)

# 6. Compilar el modelo
model.compile(optimizer=Adam(learning_rate=5e-4), loss='binary_crossentropy', metrics=['accuracy'])

# 7. Entrenar el modelo
history = model.fit(
    [X_train_numerical_scaled, X_train_text_padded],
    y_train,
    epochs=20,
    validation_split=0.1,
    batch_size=32
)

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate([X_test_numerical_scaled, X_test_text_padded], y_test)
print(f'Accuracy: {accuracy}')


Epoch 1/20


c:\Users\pabma\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


83/83 ━━━━━━━━━━━━━━━━━━━━ 10s 94ms/step - accuracy: 0.5224 - loss: 0.9466 - val_accuracy: 0.7397 - val_loss: 0.6096
Epoch 2/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 8s 91ms/step - accuracy: 0.6586 - loss: 0.7033 - val_accuracy: 0.8048 - val_loss: 0.5430
Epoch 3/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 8s 98ms/step - accuracy: 0.6774 - loss: 0.6590 - val_accuracy: 0.8219 - val_loss: 0.4725
Epoch 4/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 8s 94ms/step - accuracy: 0.7111 - loss: 0.6322 - val_accuracy: 0.8185 - val_loss: 0.4479
Epoch 5/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 8s 95ms/step - accuracy: 0.7208 - loss: 0.6040 - val_accuracy: 0.8322 - val_loss: 0.4352
Epoch 6/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 8s 91ms/step - accuracy: 0.7151 - loss: 0.5999 - val_accuracy: 0.8425 - val_loss: 0.4277
Epoch 7/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 7s 90ms/step - accuracy: 0.7468 - loss: 0.5838 - val_accuracy: 0.8322 - val_loss: 0.4228
Epoch 8/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 8s 91ms/step - accuracy: 0.7479 - loss: 0.5411 - val_accuracy: 0.8322 - val_loss: 0

In [10]:
# Guardar el modelo en formato HDF5
model.save('data/modelo_LSTM.h5')

# Guardar los datos preprocesados en un archivo pickle
with open('data/datos_preprocesados.pkl', 'wb') as file:
    pickle.dump({
        'modelo_LSTM': model,
        'X_test_numerical_scaled': X_test_numerical_scaled,
        'X_test_text_padded': X_test_text_padded,
        'y_test': y_test,
        'scaler': scaler,
        'max_length': max_length,
        'tokenizer': tokenizer
    }, file)